# Çalışmalardan
*** abs(y_pred) - abs(y_test) = fark ***
birde bu farktan ilerlemeye çalış. Fark sıfıra yakınsa demekte tahminler tutarlı.

124 model çalıştı hiç biri bire bir değerleri bulamadı. Halen gereken parametrelere ulabilmiş değilim.

True değer yok :( ------------------------->
 ----------------->Yeni Modele geçiliyor.)

* Epoch 20 civarı yetiyor, temeli görmede.


* Rebust ve learning rate sabit.

 * (1-6)-Ilk 1 LSTM 1 Dense layer. LSTM node(neuronları) artacak sürekli for döngüsü ile. ( hiç bir işe yaramıyor.)

 * (1-100)- 2 LSTM 1 Dense layer. (n = 12/13/14 + 1) (ilkinden daha iyi ancak hala 5 peşpeşe doğru tahmin yapmadı.)

 * (13-18) - 3 LSTM 1 Dense layer (n = 12/13/14 +1)







# Imports

In [1]:
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from google.colab import files
import pandas as pd
import datetime
import time
import yfinance as yf
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from sklearn.preprocessing import RobustScaler

# Data

In [23]:
# Deneme datayı alıyoruz
prices = pd.read_csv("denemelik.csv")  # Placeholder for your dataset
prices.set_index('Timestamp', inplace=True)

In [27]:
# ✅ Scale only input features, NOT "Change_up"
features_to_scale = prices.drop(columns=["Change_up"])  # Exclude target column
scaler = RobustScaler()
scaled_features = scaler.fit_transform(features_to_scale)

# ✅ Convert back to DataFrame (excluding Change_up)
scaled_prices = pd.DataFrame(scaled_features, columns=features_to_scale.columns, index=prices.index)

# ✅ Keep Change_up as is
scaled_prices["Change_up"] = prices["Change_up"]  # Restore original labels

look_back = 240  # Use 240 past candles as input
future_step = 1

X, y = [], []
for i in range(len(scaled_prices) - look_back - future_step):
    X.append(scaled_prices.iloc[i : i + look_back].values)  # Collect 240 timesteps
    y.append(scaled_prices.iloc[i + look_back + future_step - 1]["Change_up"])  # Close at t+1

# Convert to NumPy arrays
X = np.array(X)  # Shape: (samples, 240, features)
y = np.array(y)  # Shape: (samples,)

# ✅ Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# ✅ Get the datetime index for the test set
X_test_indices = scaled_prices.index[-len(X_test):]


In [28]:
# Checking the data shape
print(f" X_train shape = {X_train.shape}")  # (N, 1, 4) --> Correct shape
print(f"X_test shape = {X_test.shape}")  # (N, 1, 4) --> Correct shape
print(f" y_train shape = {y_train.shape}")  # (N, 1, 4) --> Correct shape
print(f" y_test shape = {y_test.shape}")  # (N, 1, 4) --> Correct shape

 X_train shape = (7807, 240, 8)
X_test shape = (1952, 240, 8)
 y_train shape = (7807,)
 y_test shape = (1952,)


In [36]:
y_train

array([2., 2., 2., ..., 2., 2., 2.])

# Fonks

In [29]:
def produce_predictions(model,scaled_prices):
    y_pred = model.predict(X_test)
    df_pre =  pd.DataFrame(columns=["timestamp","p_situ", "r_situ"])
    X_test_indices = scaled_prices.index[-len(X_test):]
    for i in range(len(y_pred)):
      y_pred_inv = scaler.inverse_transform(
        [[0, 0, 0, 0, 0, 0, 0, y_pred[i][0]]]
        )[0][1]
      y_test_inv = scaler.inverse_transform(
        [[0, 0, 0, 0, 0, 0, 0, y_test[i]]]
        )[0][1]
      new_row = pd.DataFrame([{"timestamp":X_test_indices[i], "p_situ": y_pred_inv, "r_situ": y_test_inv}])
      # Use pd.concat() to add the new row
      df_pre = pd.concat([df_pre, new_row], ignore_index=True)
    df_pre.to_csv(f"{model.name}_tahmin.csv")
    files.download(f"{model.name}_tahmin.csv")
    return df_pre

In [30]:
def plotting_predictions(df_pre, filename):
  # Convert timestamp to datetime (if not already done)
  df_pre['timestamp'] = pd.to_datetime(df_pre['timestamp'])
  # Create a line plot
  plt.figure(figsize=(14, 8))
  plt.plot( df_pre['timestamp'], df_pre['p_situ'], label='Predicted Price (pre_price)', color='blue', marker='o', markersize=4)
  plt.plot(df_pre['timestamp'], df_pre['r_situ'], label='Real Price (real_price)', color='red', marker='o', markersize=4)
  # Customize the axes and figure background
  ax = plt.gca()
  ax.set_facecolor('black')  # Set axes' background to black
  ax.tick_params(axis='both', colors='black')  # Set tick marks and labels to white
  ax.xaxis.label.set_color('white')  # Set x-axis label to white
  ax.yaxis.label.set_color('white')  # Set y-axis label to white
  ax.title.set_color('black')  # Set title color to white
  # Format the x-axis for minute-based grids
  ax.xaxis.set_major_locator(mdates.MinuteLocator(interval=1))  # Grid every 5 minutes
  ax.xaxis.set_major_formatter(mdates.DateFormatter('%M'))  # Format as HH:MM
  # Add labels and title
  plt.xlabel('Timestamp')
  plt.ylabel('Price')
  plt.title('Predicted vs Real Price Over Time')
  plt.legend()
  plt.grid(True, which='both', linestyle='--', linewidth=0.5, color='gray')
  # Rotate x-axis labels for better readability
  plt.xticks(rotation=45)
  # Show the plot
  plt.tight_layout()
  plt.savefig(filename, format='png')  # Change the format if needed, e.g., 'jpg', 'svg'
  files.download(filename)  # This will prompt you to download the file
  plt.show()

# For Döngüsüyle ilerleme

`2 Layers;`

**LSTM** (1-240, input_shape=(lookback(240), 9), return_sequences=True),

**Dense** (1),

**`Optimizer = Adam(0.01)`**

**`Epoch = 20`**


In [32]:
for i in range (1,10):
  model = Sequential([
      LSTM(i+62, return_sequences=True, input_shape=(240, 8), activation="tanh"),
      LSTM(i+32, return_sequences=False, activation = "tanh"),
      Dense(3, activation = "softmax" )
  ],name=f"model_{i}")
  print(model.name)
  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
                loss="sparse_categorical_crossentropy", metrics=["accuracy"])
  # Early stopping callback
  #early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
  print(f"{model.name} fitting is starting ----------------------------------->")
  model.fit(
      X_train, y_train,
      validation_split=0.1,
      epochs=20,
      batch_size=32,
      #callbacks=[early_stop]
  )
  eval = model.evaluate(X_test, y_test)
  print (f"Eval değerler = {eval} ----------------------------------->")
  y_pred = model.predict(X_test)
  # Doğru tahminleri bul
  y_pred = np.squeeze(y_pred)
  correct_predictions = y_pred == y_test
  count_values = pd.Series(correct_predictions).value_counts()
  if any(count_values.index == True):
      # Eğer True değeri varsa burası çalışır
      print("En az bir true değeri mevcut!---------------->")
      df_pre_6 = produce_predictions(model,scaled_prices)
      plotting_predictions(df_pre_6[0:30], f"{model.name}.png")
      model.save(f"{model.name}.h5")
      files.download(f"{model.name}.h5")
  else:
    print("True değer yok :( -------------------------> \n ----------------->Yeni Modele geçiliyor.")



model_1
model_1 fitting is starting ----------------------------------->
Epoch 1/20
220/220 ━━━━━━━━━━━━━━━━━━━━ 45s 187ms/step - accuracy: 0.8210 - loss: 0.5241 - val_accuracy: 0.7836 - val_loss: 0.5764
Epoch 2/20
220/220 ━━━━━━━━━━━━━━━━━━━━ 41s 185ms/step - accuracy: 0.8405 - loss: 0.4508 - val_accuracy: 0.7836 - val_loss: 0.5628
Epoch 3/20
220/220 ━━━━━━━━━━━━━━━━━━━━ 43s 194ms/step - accuracy: 0.8368 - loss: 0.4513 - val_accuracy: 0.7836 - val_loss: 0.5598
Epoch 4/20
220/220 ━━━━━━━━━━━━━━━━━━━━ 80s 186ms/step - accuracy: 0.8424 - loss: 0.4390 - val_accuracy: 0.7836 - val_loss: 0.5933
Epoch 5/20
220/220 ━━━━━━━━━━━━━━━━━━━━ 42s 192ms/step - accuracy: 0.8380 - loss: 0.4364 - val_accuracy: 0.7836 - val_loss: 0.5715
Epoch 6/20
220/220 ━━━━━━━━━━━━━━━━━━━━ 80s 185ms/step - accuracy: 0.8402 - loss: 0.4394 - val_accuracy: 0.7836 - val_loss: 0.5702
Epoch 7/20
220/220 ━━━━━━━━━━━━━━━━━━━━ 42s 189ms/step - accuracy: 0.8339 - loss: 0.4424 - val_accuracy: 0.7836 - val_loss: 0.5743
Epoch 8/20

ValueError: operands could not be broadcast together with shapes (1952,3) (1952,) 

# Deneme


In [33]:
y_pred

array([[1.2537380e-04, 3.0684877e-02, 9.6918982e-01],
       [7.1529933e-04, 4.1874066e-01, 5.8054405e-01],
       [1.8808487e-04, 1.4602330e-01, 8.5378861e-01],
       ...,
       [6.1613799e-04, 2.5337425e-01, 7.4600959e-01],
       [2.9472905e-04, 3.3131766e-01, 6.6838771e-01],
       [4.5431516e-05, 2.3463175e-01, 7.6532269e-01]], dtype=float32)

In [34]:
y_test

array([0., 2., 2., ..., 0., 0., 0.])